# Building Generative AI Applications with Gradio

## import 

In [1]:
import os
import io
import IPython.display
from PIL import Image
import base64
from transformers import pipeline
import gradio as gr 

# Image Captioning app

In [11]:
task = "image-to-text"
model = "Salesforce/blip-image-captioning-base"
image_captioner = pipeline("image-to-text") #, model = "Salesforce/blip-image-captioning-base")

No model was supplied, defaulted to ydshieh/vit-gpt2-coco-en and revision 65636df (https://huggingface.co/ydshieh/vit-gpt2-coco-en).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [12]:
img_url = "chopper.webp"
#img_url = "luffy.webp"
display(IPython.display.Image(url=img_url))
image_captioner(img_url)

'a painting of a bear with a cartoon character on it '

In [7]:
def captioner(image):
    result = image_captioner(image)
    return result[0]['generated_text']

gr.close_all()
demo = gr.Interface(fn=image_captioner,
                    inputs=[gr.Image(label="Upload image", type="pil")],
                    outputs=[gr.Textbox(label="Caption")],
                    title="Image Captioning with BLIP",
                    description="Caption any image using the BLIP model",
                    allow_flagging="never",
                    #examples=["christmas_dog.jpeg", "bird_flight.jpeg", "cow.jpeg"]
                   )

demo.launch() #share=True) #, server_port=int(os.environ['PORT1']))

Closing server running on port: 7860
Closing server running on port: 7860
Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


# Text Summarization app

In [14]:
from transformers import pipeline

get_completion = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")

def summarize(input):
    output = get_completion(input)
    return output[0]['summary_text']

In [ ]:
text = ('''The tower is 324 metres (1,063 ft) tall, about the same height
        as an 81-storey building, and the tallest structure in Paris. 
        Its base is square, measuring 125 metres (410 ft) on each side. 
        During its construction, the Eiffel Tower surpassed the Washington 
        Monument to become the tallest man-made structure in the world,
        a title it held for 41 years until the Chrysler Building
        in New York City was finished in 1930. It was the first structure 
        to reach a height of 300 metres. Due to the addition of a broadcasting 
        aerial at the top of the tower in 1957, it is now taller than the 
        Chrysler Building by 5.2 metres (17 ft). Excluding transmitters, the 
        Eiffel Tower is the second tallest free-standing structure in France 
        after the Millau Viaduct.''')

summarize(text)

In [15]:
# import gradio as gr
# def summarize(input):
#     output = get_completion(input)
#     return output[0]['summary_text']
    
demo = gr.Interface(fn=summarize, inputs="text", outputs="text")
demo.launch() #share=True, server_port=int(os.environ['PORT1']))

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


In [16]:
import gradio as gr

# def summarize(input):
#     output = get_completion(input)
#     return output[0]['summary_text']

gr.close_all()
demo = gr.Interface(fn=summarize, 
                    inputs=[gr.Textbox(label="Text to summarize", lines=6)],
                    outputs=[gr.Textbox(label="Result", lines=3)],
                    title="Text summarization with distilbart-cnn",
                    description="Summarize any text using the `shleifer/distilbart-cnn-12-6` model under the hood!"
                   )
demo.launch() #share=True, server_port=int(os.environ['PORT2']))

Closing server running on port: 7860
Closing server running on port: 7861
Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [ ]:
gr.close_all()

# Named Entity Recognition app

In [ ]:
from transformers import pipeline

get_completion = pipeline("ner", model="dslim/bert-base-NER")

def ner(input):
    output = get_completion(input)
    return {"text": input, "entities": output}

In [ ]:
# def ner(input):
#     output = get_completion(input, parameters=None, ENDPOINT_URL=API_URL)
#     return {"text": input, "entities": output}

gr.close_all()
demo = gr.Interface(fn=ner,
                    inputs=[gr.Textbox(label="Text to find entities", lines=2)],
                    outputs=[gr.HighlightedText(label="Text with entities")],
                    title="NER with dslim/bert-base-NER",
                    description="Find entities using the `dslim/bert-base-NER` model under the hood!",
                    allow_flagging="never",
                    #Here we introduce a new tag, examples, easy to use examples for your application
                    examples=["My name is Andrew and I live in California", "My name is Poli and work at HuggingFace"])
demo.launch() #share=True, server_port=int(os.environ['PORT3']))

In [ ]:
gr.close_all()

# Image Generation App

# Describe-and-Generate game

# Chat App